<font size=12, color=blue>  <center> <b>Aprendizaje Profundo

<font size=6, color=blue>  <center> <b> (Una concisa introducción)

<font size=6, color=green>  <center> <b>Ejemplo práctico en laboratorio

#### Práctica sobre aprendizaje profundo en la que se mostrarán algunos ejemplos de este tipo de técnicas, como los autocodificadores, la técnica de "drop-out" o las redes convolucionales (CNNs)

<font size=4, color=blue> Magic para representaciones on-line

In [2]:
%matplotlib inline

<font size=4, color=blue> Se importan algunas librerías básicas

In [3]:
#import sys
#import os
#import time
#import scipy.io as sio

import numpy as np
import matplotlib.pyplot as plt
import metodosDL as mdl

ModuleNotFoundError: No module named 'metodosDL'

<font size=4, color=blue> A continuación se cargan los datos de la base de datos MNIST (números manuscritos)

In [4]:
import gzip

def carga_MNIST_patrones(fichero):
        # Lectura de los datos MNIST con el formato de Yann LeCun
        with gzip.open(fichero, 'rb') as f:
            patrones = np.frombuffer(f.read(), np.uint8, offset=16)
        # Los patrones son vectores que contienen imágenes 2D en escala de grises
        # patrones = patrones.reshape(-1, 28, 28)
        # Se agrupan los patrones por filas (cada patrón es una fila, cada columna un pixel)
        patrones = patrones.reshape(-1,784)
        # Conversión de bytes a float32 en el rango [0,1].
        return patrones / np.float32(256)
    
def carga_MNIST_etiquetas(fichero):
        # Lectura de las etiquetas en el formato de Yann LeCun
        with gzip.open(fichero, 'rb') as f:
            etiquetas = np.frombuffer(f.read(), np.uint8, offset=8)
        # Las etiquetas son valores enteros
        return etiquetas

xTrain = carga_MNIST_patrones('train-images-idx3-ubyte.gz')
yTrain = carga_MNIST_etiquetas('train-labels-idx1-ubyte.gz')
xTest = carga_MNIST_patrones('t10k-images-idx3-ubyte.gz')
yTest = carga_MNIST_etiquetas('t10k-labels-idx1-ubyte.gz') 

# Los datos de entrenamiento se dividen en conjuntos de entrenamiento y de validación
xTrain, xVal = xTrain[:-10000], xTrain[-10000:]
yTrain, yVal = yTrain[:-10000], yTrain[-10000:]

# Se transponen los datos para tener los patrones por columnas
xTrain=xTrain.T
xTest=xTest.T
xVal=xVal.T

print("Las imágenes se agrupan en una matriz, donde cada columna es una imagen con 784 píxeles (28x28)\r\n")
print("Tamaño de la matriz de patrones de entrenamiento: %s x %s"%(xTrain.shape))
print("Tamaño de la matriz de patrones de validación: %s x %s"%(xVal.shape))
print("Tamaño de la matriz de patrones de test: %s x %s"%(xTest.shape))

FileNotFoundError: [Errno 2] No such file or directory: 'train-images-idx3-ubyte.gz'

<font size=4, color=blue> Se representa un dígito (ejemplo de los datos)

In [ ]:
indice=0;
imagen = xTrain[:,indice]
imagen=imagen.reshape(28,28)
etiqueta=yTrain[indice]
fig, eje = plt.subplots(figsize=(6, 6))
eje.imshow(imagen, cmap='gray')
eje.set_title('{label}'.format(label=etiqueta))

<font size=4, color=blue> Se representan ahora ejemplos de cada clase (el primero de cada clase en el conjunto de entrenamiento)

In [ ]:
indices=[]
# Se busca la posición en la que se encuentra el primer patrón de cada clase
for k in range(10):
    indNum = [i for (i, val) in enumerate(yTrain[0:30]) if val == int(k)]
    indices.append(indNum[0])

fig, ejes = plt.subplots(1, 10, figsize=(12, 1.5), subplot_kw={'xticks': [], 'yticks': []})
fig.subplots_adjust(hspace=0.3, wspace=0.05)

for eje, k in zip(ejes.flat, indices):
    imagen = xTrain[:,k]
    imagen=imagen.reshape(28,28)
    etiqueta = yTrain[k]
    eje.imshow(imagen, cmap='gray')
    eje.set_title('{label}'.format(label=etiqueta))


<font size=4, color=blue> Por cuestiones de carga computacional se va a utilizar una menor resolución diezmando


In [ ]:
indDown=[]
#pixeles=np.linspace(0,27,num=28)  # para mantener 28 x 28
#pixeles=np.linspace(0,26,num=14)  # para diezmar a 14 x 14
pixeles=np.linspace(0,27,num=10)  # para diezmar a 10 x 10
for ka in pixeles:
    for kb in pixeles:
        indDown.append(int(ka)*28+int(kb))       

xTrainD=xTrain[indDown,:]
xValD=xVal[indDown,:]
xTestD=xTest[indDown,:]

fig, ejes = plt.subplots(1, 10, figsize=(12, 1.5), subplot_kw={'xticks': [], 'yticks': []})
fig.subplots_adjust(hspace=0.3, wspace=0.05)

for eje, k in zip(ejes.flat, indices):
    imagen = xTrainD[:,k]
    imagen=imagen.reshape(len(pixeles),len(pixeles))
    etiqueta = yTrain[k]
    eje.imshow(imagen, cmap='gray')
    eje.set_title('{label}'.format(label=etiqueta))

<font size=4, color=blue> Para reducir la carga computacional se realiza un muestreo, reduciendo el número de imágenes de entrenamiento por un factor "*factorS*"

In [ ]:
factorS=25    # Factor de muestreo (el número de muestras se divide por este factor)
indTrainS=range(0,len(yTrain),factorS)
xTrainS=xTrainD[:,indTrainS]
yTrainS=yTrain[indTrainS]

print("Tamaño de la matriz de patrones de entrenamiento: %s x %s"%(xTrainS.shape))
print("Tamaño de la matriz de patrones de validación: %s x %s"%(xValD.shape))
print("Tamaño de la matriz de patrones de test: %s x %s"%(xTestD.shape))


<font size=4, color=blue>Habitualmente, para una clasificación multiclase se utiliza una red con M salidas, una por cada clase. La referencia de la salida para el patrón de la clase *i*-ésima es un vector con M-1 ceros y un uno en la posición *i*-ésima. Se crea por tanto una matriz con los vectores de referencia para cada patrón


In [ ]:
eTrainS=np.zeros((10,len(yTrainS)))
for k in range(len(yTrainS)):
    aux=yTrainS[k]
    eTrainS[aux,k]=1
print("Tamaño de la matriz de etiquetas de referencia: %s x %s"%(eTrainS.shape)) 

<font size=4, color=blue>Se muestra la referencia de la salida para un cierto patrón

In [ ]:
indice=0
print("Patrón de indice %d - Clase = %d"%(indice,yTrainS[indice]))
print(eTrainS[:,indice])
imagen = xTrainS[:,indice]
imagen=imagen.reshape(len(pixeles),len(pixeles))
etiqueta=yTrainS[indice]
fig, eje = plt.subplots(figsize=(3, 3))
eje.imshow(imagen, cmap='gray')
eje.set_title('{label}'.format(label=etiqueta))

<br>
<font size=6, color=green>Entrenamiento de una red neuronal no profunda 

<font size=4, color=blue>En primer lugar se definen los parámetros de la red, que es un perceptrón multicapa (MLP) de una única capa oculta

In [ ]:
fCoste='entropia'   # Tipo de función de coste ('mmse', 'entropia' ,'wmmse')
opt='gradiente'     # Tipo de optimizador ('gradiente', 'momento')
tAct=[1,4]          # Tipos de activación de las neuronas de la capa oculta y de la capa de salida
paso=[1e-3]         # Parametro de paso de la actualización por gradiente
Nn=125              # Número de neuronas de la capa oculta 
Niter=250           # Número de iteraciones para la actualización por gradiente
Nbatch=0            # Número de muestras del mini-batch
Nep=[Niter,Nbatch]  # Parámetros agrupados

(Ne,Np)=xTrainS.shape
(Ns,Nada)=eTrainS.shape

# Inicialización de los pesos de la capa oculta y de la capa de salida
inicializacionExterna=False
if inicializacionExterna:
    woini = 0.2*np.random.rand(Nn,Ne+1)-0.1
    wsini = 0.2*np.random.rand(Ns,Nn+1)-0.1
    W=[woini,wsini]
else:
    W=mdl.entrena_mlp(xTrainS,eTrainS,[Nn,[]],[0,0])[0]
    woini=W[0]
    wsini=W[1]
    

<font size=4, color=blue>Se realiza el entrenamiento del MLP y se representa la evolución del error cuadrático entre la salida de la red y los vectores de referencia para cada patrón

In [ ]:
(W, paso, coste) = mdl.entrena_mlp(xTrainS,eTrainS,W,Nep,fCoste=fCoste,optimizador=opt,tAct=tAct)[0:3]

font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
plt.plot(range(Niter+1),coste)
plt.title('Evolucion del coste a minimizar', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('Coste : %s' % (fCoste), fontdict=font)

<font size=4, color=blue>Se obtiene la salida de la red para los patrones de entrenamiento, los de validación y los de test y se estiman la probabilidad de acierto obtenidas sobre los 3 conjuntos de datos

In [ ]:
(eTrain,xOculta)=mdl.mlp(xTrainS,W,tAct)
(eVal,xOculta)=mdl.mlp(xValD,W,tAct)
(eTest,xOculta)=mdl.mlp(xTestD,W,tAct)

yTrainE=eTrain.argmax(axis=0)
yValE=eVal.argmax(axis=0)
yTestE=eTest.argmax(axis=0)

diferencia=yTrainE-yTrainS
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
PaTrain=(100.0*len(indAcierto))/len(yTrainS)

diferencia=yValE-yVal
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
PaVal=(100.0*len(indAcierto))/len(yVal)

diferencia=yTestE-yTest
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
indFallo = [i for (i, val) in enumerate(diferencia) if val != 0]
PaTest=(100.0*len(indAcierto))/len(yTest)

print("Probabilidad de acierto (Entrenamiento): %s"%(PaTrain))
print("Probabilidad de acierto (Validación): %s"%(PaVal))
print("Probabilidad de acierto (Test): %s"%(PaTest))

<font size=4, color=blue>Se pueden ver algunos de los ejemplos de patrones clasificados erróneamente

In [ ]:
indiceFallo=0
indice=indFallo[indiceFallo];
imagen = xTestD[:,indice]
imagen=imagen.reshape(10,10)
etiqueta=yTest[indice]
fig, eje = plt.subplots(figsize=(3, 3))
#eje.title('{label}'.format(label=etiqueta))
eje.imshow(imagen, cmap='gray')
print("La etiqueta correcta es %s y la red predice %s"%(etiqueta,yTestE[indice]))

<font size=4, color=blue> Se pueden representar los coeficientes de una neurona específica

In [ ]:
indice=0
imagen = W[0][indice,0:Ne]
resI=int(np.sqrt(Ne))
imagen=imagen.reshape(resI,resI)
fig, eje = plt.subplots(figsize=(6, 6))
eje.set_title('Pesos de la neurona {label}'.format(label=indice+1))
eje.imshow(imagen, cmap='gray')

<font size=4, color=blue> Se representan los pesos obtenidos en algunas de las neuronas

In [ ]:
indices=range(100)
resI=int(np.sqrt(Ne))

fig, ejes = plt.subplots(5, 20, figsize=(19, 5), subplot_kw={'xticks': [], 'yticks': []})
fig.subplots_adjust(hspace=0.3, wspace=0.05)

for eje, k in zip(ejes.flat, indices):
    imagen = W[0][k,0:Ne]
    imagen=imagen.reshape(resI,resI)
    eje.imshow(imagen, cmap='gray')

<font size=4, color=blue> Se representa la evolución de la tasa de acierto con las iteraciones

In [ ]:
W=[woini,wsini]
NiterPaso=50
NrepBucle=60
evoPa=np.zeros((3,NrepBucle))
for kbucle in range(NrepBucle):
    (W, paso, coste) = mdl.entrena_mlp(xTrainS,eTrainS,W,[NiterPaso,Nbatch],fCoste=fCoste,optimizador=opt,tAct=tAct)[0:3]
    (eTrain,xOculta)=mdl.mlp(xTrainS,W,tAct)
    (eVal,xOculta)=mdl.mlp(xValD,W,tAct)
    (eTest,xOculta)=mdl.mlp(xTestD,W,tAct)

    yTrainE=eTrain.argmax(axis=0)
    yValE=eVal.argmax(axis=0)
    yTestE=eTest.argmax(axis=0)

    diferencia=yTrainE-yTrainS
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    PaTrain=(100.0*len(indAcierto))/len(yTrainS)

    diferencia=yValE-yVal
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    PaVal=(100.0*len(indAcierto))/len(yVal)

    diferencia=yTestE-yTest
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    indFallo = [i for (i, val) in enumerate(diferencia) if val != 0]
    PaTest=(100.0*len(indAcierto))/len(yTest)
    evoPa[:,kbucle]=[PaTrain,PaVal,PaTest]
    
font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
ejeIter=NiterPaso*np.linspace(1,NrepBucle,NrepBucle)
plt.plot(ejeIter,evoPa[0,:],ejeIter,evoPa[1,:],ejeIter,evoPa[2,:])
plt.title('Evolucion de la Prob. Acierto', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('Prob. Acierto (%)', fontdict=font)

print("Entrenamiento (azul) - Validación (Naranja) - Test (verde)")
print(" ")
print("Max. Pa (Entrenamiento): %3.2f (Iter. %3.2f)"%(np.max(evoPa[0,:]),NiterPaso*np.argmax(evoPa[0,:])))
print("Max. Pa (Validación): %3.2f (Iter. %3.2f)"%(np.max(evoPa[1,:]),NiterPaso*np.argmax(evoPa[1,:])))
print("Max. Pa (Test) : %3.2f (Iter. %3.2f)"%(np.max(evoPa[2,:]),NiterPaso*np.argmax(evoPa[2,:])))
print("Resultado con validación (Test): %s"%(evoPa[2,np.argmax(evoPa[1,:])]))

<br>
<font size=6, color=green> Entrenamiento de una red profunda
<font size=4, color=blue> Se entrena en este caso un MLP de 4 capas ocultas y una capa de salida

In [ ]:
fCoste='entropia' # Tipo de función de coste ('mmse', 'entropia' ,'wmmse')
opt='gradiente'   # Tipo de optimizador ('gradiente', 'momento')
tAct=[1,1,1,1,4]  # Tipos de activación de las neuronas de las capas oculta y de la capa de salida
paso=[1e-3]       # Parametro de paso de la actualización por gradiente
Nn1=125           # Número de neuronas de la primera capa oculta
Nn2=125           # Número de neuronas de la segunda capa oculta
Nn3=125           # Número de neuronas de la tercera capa oculta 
Nn4=125           # Número de neuronas de la cuarta capa oculta 
Niter=250         # Número de iteraciones para la actualización por gradiente
Nbatch=250        # Número de muestras del mini-batch
Nep=[Niter,Nbatch]

(Np,Ne)=xTrainS.shape
(Ns,Nada)=eTrainS.shape

# Inicialización de los pesos de la capa oculta y de la capa de salida
inicializacionExterna=False
if inicializacionExterna:
    wo1ini = 0.2*np.random.rand(Nn1,Ne+1)-0.1
    wo2ini = 0.2*np.random.rand(Nn2,Nn1+1)-0.1
    wo3ini = 0.2*np.random.rand(Nn3,Nn2+1)-0.1
    wo4ini = 0.2*np.random.rand(Nn4,Nn3+1)-0.1
    wsini = 0.2*np.random.rand(Ns,Nn4+1)-0.1
    W=[wo1ini,wo2ini,wo3ini,wo4ini,wsini]
else:
    W=mdl.entrena_mlp(xTrainS,eTrainS,[Nn1,Nn2,Nn3,Nn4,[]],[0,0])[0]
    wo1ini=W[0]
    wo2ini=W[1]
    wo3ini=W[2]
    wo4ini=W[3]
    wsini=W[4]
    

In [ ]:
(W, paso, coste) = mdl.entrena_mlp(xTrainS,eTrainS,W,Nep,fCoste=fCoste,optimizador=opt,tAct=tAct,paso=paso)[0:3]

font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
plt.plot(range(Niter+1),coste)
plt.title('Evolucion del coste a minimizar', fontdict=font)
plt.xlabel('Epoca', fontdict=font)
plt.ylabel('Coste : %s' % (fCoste), fontdict=font)

In [ ]:
eTrain=mdl.mlp(xTrainS,W,tAct)[0]
eVal=mdl.mlp(xValD,W,tAct)[0]
eTest=mdl.mlp(xTestD,W,tAct)[0]

yTrainE=eTrain.argmax(axis=0)
yValE=eVal.argmax(axis=0)
yTestE=eTest.argmax(axis=0)

diferencia=yTrainE-yTrainS
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
PaTrain=(100.0*len(indAcierto))/len(yTrainS)

diferencia=yValE-yVal
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
PaVal=(100.0*len(indAcierto))/len(yVal)

diferencia=yTestE-yTest
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
indFallo = [i for (i, val) in enumerate(diferencia) if val != 0]
PaTest=(100.0*len(indAcierto))/len(yTest)

print("Probabilidad de acierto (Entrenamiento): %s"%(PaTrain))
print("Probabilidad de acierto (Validación): %s"%(PaVal))
print("Probabilidad de acierto (Test): %s"%(PaTest))

<font size=4, color=blue> Se representa la evolución de la tasa de acierto con las iteraciones

In [ ]:
W=[wo1ini,wo2ini,wo3ini,wo4ini,wsini]
NiterPaso=50
NrepBucle=60
evoPa=np.zeros((3,NrepBucle))
for kbucle in range(NrepBucle):
    (W, paso, coste) = mdl.entrena_mlp(xTrainS,eTrainS,W,[NiterPaso,Nbatch],fCoste=fCoste,optimizador=opt,tAct=tAct)[0:3]
    (eTrain,xOculta)=mdl.mlp(xTrainS,W,tAct)
    (eVal,xOculta)=mdl.mlp(xValD,W,tAct)
    (eTest,xOculta)=mdl.mlp(xTestD,W,tAct)

    yTrainE=eTrain.argmax(axis=0)
    yValE=eVal.argmax(axis=0)
    yTestE=eTest.argmax(axis=0)

    diferencia=yTrainE-yTrainS
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    PaTrain=(100.0*len(indAcierto))/len(yTrainS)

    diferencia=yValE-yVal
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    PaVal=(100.0*len(indAcierto))/len(yVal)

    diferencia=yTestE-yTest
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    indFallo = [i for (i, val) in enumerate(diferencia) if val != 0]
    PaTest=(100.0*len(indAcierto))/len(yTest)
    evoPa[:,kbucle]=[PaTrain,PaVal,PaTest]
    
font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
ejeIter=NiterPaso*np.linspace(1,NrepBucle,NrepBucle)
plt.plot(ejeIter,evoPa[0,:],ejeIter,evoPa[1,:],ejeIter,evoPa[2,:])
plt.title('Evolucion de la Prob. Acierto', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('Prob Acierto (%)', fontdict=font)

print("Entrenamiento (azul) - Validación (Naranja) - Test (verde)")
print(" ")
print("Max. Pa (Entrenamiento): %3.2f (Iter. %3.2f)"%(np.max(evoPa[0,:]),NiterPaso*np.argmax(evoPa[0,:])))
print("Max. Pa (Validación): %3.2f (Iter. %3.2f)"%(np.max(evoPa[1,:]),NiterPaso*np.argmax(evoPa[1,:])))
print("Max. Pa (Test) : %3.2f (Iter. %3.2f)"%(np.max(evoPa[2,:]),NiterPaso*np.argmax(evoPa[2,:])))
print("Resultado con validación (Test): %s"%(evoPa[2,np.argmax(evoPa[1,:])]))

<br>
<font size=6, color=green>Entrenamiento de un autocodificador sin ruido

<font size=4, color=blue>Se entrenará ahora un autocodificador para una red MLP

In [ ]:
# Se inicializan los parámetros del codificador y de su entrenamiento
tAct=[1,2]         # Tipos de activación de las neuronas de la capa oculta y de la capa de salida
paso=[1e-3]        # Parametro de paso de la actualización por gradiente
Nn=125             # Número de neuronas de la capa oculta 
Niter=250          # Número de iteraciones para la actualización por gradiente

(Ne,Np)=xTrainS.shape
(Ns,Nada)=eTrainS.shape
# Inicialización de los pesos de la capa oculta y de la capa de salida
weini = 0.2*np.random.rand(Nn,Ne+1)-0.1
wrini = 0.2*np.random.rand(Ne,Nn+1)-0.1

we1=weini
wr1=wrini

In [ ]:
(we1, wr1, coste, paso)=mdl.entrena_AE(xTrainS,we1,wr1,Niter,tAct,paso)
# En realidad, un autocodificador sin ruido se puede ver como una red neuronal en la que la salida de referencia es
# igual a los patrones de entrada
#
# W=[we1,wr1]
# (W, paso, coste)=mdl.entrena_mlp(xTrainS,xTrainS,W,[Niter,0],tAct=tAct,paso=paso)[0:3]
# we1=W[0]
# wr1=W[1]

font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
plt.plot(range(Niter+1),coste)
plt.title('Evolucion del MSE para codificador sin ruido', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('MSE', fontdict=font)

<font size=4, color=blue>Se representan las reconstrucciones obtenidas a partir del codificador

In [ ]:
(xTrainDrec,xTrainE1)=mdl.mlp(xTrainD,[we1,wr1],tAct)

indices=[]

for k in range(10):
    indNum = [i for (i, val) in enumerate(yTrain[0:30]) if val == int(k)]
    indices.append(indNum[0])

figA, ejesA = plt.subplots(1, 10, figsize=(12, 3), subplot_kw={'xticks': [], 'yticks': []})
figA.subplots_adjust(hspace=0.3, wspace=0.05)
figB, ejesB = plt.subplots(1, 10, figsize=(12, 3), subplot_kw={'xticks': [], 'yticks': []})
figB.subplots_adjust(hspace=0.3, wspace=0.05)

for eje, k in zip(ejesA.flat, indices):
    imagen = xTrainD[:,k]
    imagen=imagen.reshape(len(pixeles),len(pixeles))
    etiqueta = yTrain[k]
    eje.imshow(imagen, cmap='gray')
    eje.set_title('a')
    eje.set_title('{label}'.format(label=etiqueta))
for eje, k in zip(ejesB.flat, indices):    
    imagen = xTrainDrec[:,k]
    imagen=imagen.reshape(len(pixeles),len(pixeles))
    eje.imshow(imagen, cmap='gray')

<font size=4, color=blue>Se representan los pesos de algunas neuronas de la capa oculta (codificador)

In [ ]:
indice=0
imagen = we1[indice,0:Ne]
resI=int(np.sqrt(Ne))
imagen=imagen.reshape(resI,resI)
fig, eje = plt.subplots(figsize=(6, 6))
eje.set_title('Pesos de la neurona {label} (AE)'.format(label=indice+1))
eje.imshow(imagen, cmap='gray')

In [ ]:
indices=range(100)
resI=int(np.sqrt(Ne))

fig, ejes = plt.subplots(5, 20, figsize=(19, 5), subplot_kw={'xticks': [], 'yticks': []})
fig.subplots_adjust(hspace=0.3, wspace=0.05)

for eje, k in zip(ejes.flat, indices):
    imagen = we1[k,0:Ne]
    imagen=imagen.reshape(resI,resI)
    eje.imshow(imagen, cmap='gray')

<br>
<font size=6, color=green>Entrenamiento de un autocodificador con ruido
<font size=4, color=blue> Ahora se entrena un autocodificador con ruido en la entrada

In [ ]:
# Se inicializan los parámetros del codificador y de su entrenamiento
tAct=[1,2]       # Tipos de activación de las neuronas de la capa oculta y de la capa de salida
paso=[1e-3]      # Parametro de paso de la actualización por gradiente
Nn=125           # Número de neuronas de la capa oculta 
Niter=250        # Número de iteraciones para la actualización por gradiente
pRuido=0.25      # Probabilidad de poner a cero una cierta entrada

# Inicialización de los pesos de la capa oculta y de la capa de salida
# weini = 0.2*np.random.rand(Nn,Ne+1)-0.1
# wdini = 0.2*np.random.rand(Ne,Nn+1)-0.1

we1n=weini
wr1n=wrini

In [ ]:
(we1n, wr1n, coste, paso)=mdl.entrena_DAE(xTrainS,we1n,wr1n,Niter,tAct,paso,pRuido)

font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
plt.plot(range(Niter+1),coste)
plt.title('Evolucion del MSE para un codificador con ruido', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('MSE', fontdict=font)

<font size=4, color=blue>Se representan las reconstrucciones obtenidas a partir del codificador

In [ ]:
(xTrainDrec,xTrainE1n)=mdl.mlp(xTrainD,[we1n,wr1n],tAct)

indices=[]

for k in range(10):
    indNum = [i for (i, val) in enumerate(yTrain[0:30]) if val == int(k)]
    indices.append(indNum[0])

figA, ejesA = plt.subplots(1, 10, figsize=(12, 3), subplot_kw={'xticks': [], 'yticks': []})
figA.subplots_adjust(hspace=0.3, wspace=0.05)
figB, ejesB = plt.subplots(1, 10, figsize=(12, 3), subplot_kw={'xticks': [], 'yticks': []})
figB.subplots_adjust(hspace=0.3, wspace=0.05)

for eje, k in zip(ejesA.flat, indices):
    imagen = xTrainD[:,k]
    imagen=imagen.reshape(len(pixeles),len(pixeles))
    etiqueta = yTrain[k]
    eje.imshow(imagen, cmap='gray')
    eje.set_title('a')
    eje.set_title('{label}'.format(label=etiqueta))
for eje, k in zip(ejesB.flat, indices):    
    imagen = xTrainDrec[:,k]
    imagen=imagen.reshape(len(pixeles),len(pixeles))
    eje.imshow(imagen, cmap='gray')

<font size=4, color=blue>Se representan los pesos de algunas neuronas de la capa oculta (codificador)

In [ ]:
indice=0
imagen = we1n[indice,0:Ne]
resI=int(np.sqrt(Ne))
imagen=imagen.reshape(resI,resI)
fig, eje = plt.subplots(figsize=(6, 6))
eje.set_title('Pesos de la neurona {label} (DAE)'.format(label=indice+1))
eje.imshow(imagen, cmap='gray')

In [ ]:
indices=range(100)
resI=int(np.sqrt(Ne))

fig, ejes = plt.subplots(5, 20, figsize=(19, 5), subplot_kw={'xticks': [], 'yticks': []})
fig.subplots_adjust(hspace=0.3, wspace=0.05)

for eje, k in zip(ejes.flat, indices):
    imagen = we1n[k,0:Ne]
    imagen=imagen.reshape(resI,resI)
    eje.imshow(imagen, cmap='gray')

<br>
<font size=6, color=green>Autocodificadores "apilados"
<font size=4, color=blue>Se utilizarán ahora autoencoders para obtener los parámetros de las capas ocultas de una red profunda. A esta alternativa se le denomina autocodificadores apilados, o en inglés (Stacked Denoising Autoencoders)

In [ ]:
# Se comienza por definir los parámetros básicos del sistema
tActAE1=[1,2]     # Tipos de activación de las neuronas de la capas oculta y de la capa de salida del AE-1
tActAE2=[1,1]     # Tipos de activación de las neuronas de la capas oculta y de la capa de salida del AE-2
tActAE3=[1,1]     # Tipos de activación de las neuronas de la capas oculta y de la capa de salida del AE-3
paso=[1e-3]       # Parametro de paso de la actualización por gradiente
Nn1=125           # Número de neuronas de la primera capa oculta
Nn2=125           # Número de neuronas de la segunda capa oculta
Nn3=125           # Número de neuronas de la tercera capa oculta 
Niter=250         # Número de iteraciones para la actualización por gradiente
pRuido=0.25       # Probabilidad de anular una de las entradas

(Ne,Np)=xTrainS.shape
# Inicialización de los pesos de los autocodificadores
we1ini = 0.2*np.random.rand(Nn1,Ne+1)-0.1
we2ini = 0.2*np.random.rand(Nn2,Nn1+1)-0.1
we3ini = 0.2*np.random.rand(Nn3,Nn2+1)-0.1
# Inicialización de los pesos de reconstrucción
wr1ini = 0.2*np.random.rand(Ne,Nn1+1)-0.1
wr2ini = 0.2*np.random.rand(Nn1,Nn2+1)-0.1
wr3ini = 0.2*np.random.rand(Nn2,Nn3+1)-0.1
# Inicialización de los pesos
we1=we1ini
we2=we2ini
we3=we3ini
wr1=wr1ini
wr2=wr2ini
wr3=wr3ini

<font size=4, color=blue> Se entrena el primer AE

In [ ]:
(we1, wr1, coste, paso)=mdl.entrena_DAE(xTrainS,we1,wr1,Niter,tActAE1,paso,pRuido)

font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
plt.plot(range(Niter+1),coste)
plt.title('Evolucion del MSE para el primer AE', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('MSE', fontdict=font)

<font size=4, color=blue>Se realiza la proyección obtenida con el primer autocodificador

In [ ]:
xTrainSE1=mdl.mlp(xTrainS,[we1,wr1],tActAE1)[1][0]
xTestDE1=mdl.mlp(xTestD,[we1,wr1],tActAE1)[1][0]
xValDE1=mdl.mlp(xValD,[we1,wr1],tActAE1)[1][0]

<font size=4, color=blue> Se entrena el segundo AE

In [ ]:
(we2, wr2, coste, paso)=mdl.entrena_DAE(xTrainSE1,we2,wr2,Niter,tActAE2,paso,pRuido)

font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
plt.plot(range(Niter+1),coste)
plt.title('Evolucion del MSE para el segundo AE', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('MSE', fontdict=font)

<font size=4, color=blue>Se realiza la proyección obtenida con el segundo autocodificador

In [ ]:
xTrainSE2=mdl.mlp(xTrainSE1,[we2,wr2],tActAE2)[1][0]
xTestDE2=mdl.mlp(xTestDE1,[we2,wr2],tActAE2)[1][0]
xValDE2=mdl.mlp(xValDE1,[we2,wr2],tActAE2)[1][0]

<font size=4, color=blue>Se entrena el tercer AE

In [ ]:
(we3, wr3, coste, paso)=mdl.entrena_DAE(xTrainSE2,we3,wr3,Niter,tActAE3,paso,pRuido)

font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
plt.plot(range(Niter+1),coste)
plt.title('Evolucion del MSE para el tercer AE', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('MSE', fontdict=font)

<font size=4, color=blue> Se realiza la proyección obtenida con el tercer autocodificador

In [ ]:
xTrainSE3=mdl.mlp(xTrainSE2,[we3,wr3],tActAE3)[1][0]
xTestDE3=mdl.mlp(xTestDE2,[we3,wr3],tActAE3)[1][0]
xValDE3=mdl.mlp(xValDE2,[we3,wr3],tActAE3)[1][0]

<font size=4, color=blue> Se entrena ahora la etapa de salida (una capa oculta y una de salida)

In [ ]:
fCoste='entropia'   # Tipo de función de coste ('mmse', 'entropia' ,'wmmse')
opt='gradiente'     # Tipo de optimizador ('gradiente', 'momento')
tActFin=[1,4]       # Tipos de activación de las neuronas de la capa oculta y de la capa de salida
paso=1e-3           # Parametro de paso de la actualización por gradiente
NnFin=150           # Número de neuronas de la capa oculta 
Niter=250           # Número de iteraciones para la actualización por gradiente
Nbatch=250            # Número de patrones del mini-batch 
Nep=[Niter,Nbatch]

(Ne,Np)=xTrainSE3.shape
(Ns,Nada)=eTrainS.shape

# Inicialización de los pesos de la capa oculta y de la capa de salida
inicializacionExterna=True
if inicializacionExterna:
    woini = 0.2*np.random.rand(NnFin,Ne+1)-0.1
    wsini = 0.2*np.random.rand(Ns,NnFin+1)-0.1
    Wfin=[woini,wsini]
else:
    Wfin=mdl.entrena_mlp(xTrainS,eTrainS,[NnFin,[]],[0,0])[0]
    woini=Wfin[0]
    wsini=Wfin[1]


In [ ]:
#Niter=3000
#Nep=[Niter,250]
(Wfin, paso, coste) = mdl.entrena_mlp(xTrainSE3,eTrainS,Wfin,Nep,fCoste=fCoste,optimizador=opt,tAct=tActFin)[0:3]

font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
plt.plot(range(Niter+1),coste)
plt.title('Evolucion del coste a minimizar', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('Coste : %s' % (fCoste), fontdict=font)

<font size=4, color=blue> Se estiman las prestaciones obtenidas con esta etapa

In [ ]:
eTrain=mdl.mlp(xTrainSE3,Wfin,tActFin)[0]
eVal=mdl.mlp(xValDE3,Wfin,tActFin)[0]
eTest=mdl.mlp(xTestDE3,Wfin,tActFin)[0]

yTrainE=eTrain.argmax(axis=0)
yValE=eVal.argmax(axis=0)
yTestE=eTest.argmax(axis=0)

diferencia=yTrainE-yTrainS
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
PaTrain=(100.0*len(indAcierto))/len(yTrainS)

diferencia=yValE-yVal
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
PaVal=(100.0*len(indAcierto))/len(yVal)

diferencia=yTestE-yTest
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
indFallo = [i for (i, val) in enumerate(diferencia) if val != 0]
PaTest=(100.0*len(indAcierto))/len(yTest)

print("Probabilidad de acierto (Entrenamiento): %s"%(PaTrain))
print("Probabilidad de acierto (Validación): %s"%(PaVal))
print("Probabilidad de acierto (Test): %s"%(PaTest))

<font size=4, color=blue>Se realiza el ajuste fino sobre la red global

In [ ]:
# Se agrupan en una lista los pesos de cada capa y las funciones de activación de cada capa
Wtotal=[we1,we2,we3,Wfin[0],Wfin[1]] 
paso=[1e-3]
tActTotal=np.array([tActAE1[0], tActAE2[0], tActAE3[0], tActFin[0], tActFin[1]])
Niter=50            # Número de iteraciones para la actualización por gradiente
Nbatch=0            # Número de patrones del mini-batch 
Nep=[Niter,Nbatch]

In [ ]:
# Se entrena la red global con las 4 capas ocultas y 1 capa de salida
Niter=500
Nep=[Niter,250]
(Wtotal, paso, coste) = mdl.entrena_mlp(xTrainS,eTrainS,Wtotal,Nep,fCoste=fCoste,
                                        optimizador=opt,tAct=tActTotal,paso=paso)[0:3]

font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
plt.plot(range(Niter+1),coste)
plt.title('Evolucion del coste a minimizar', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('Coste : %s' % (fCoste), fontdict=font)

<font size=4, color=blue> Se estiman las prestaciones obtenidas con la red completa

In [ ]:
eTrain=mdl.mlp(xTrainS,Wtotal,tActTotal)[0]
eVal=mdl.mlp(xValD,Wtotal,tActTotal)[0]
eTest=mdl.mlp(xTestD,Wtotal,tActTotal)[0]

yTrainE=eTrain.argmax(axis=0)
yValE=eVal.argmax(axis=0)
yTestE=eTest.argmax(axis=0)

diferencia=yTrainE-yTrainS
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
PaTrain=(100.0*len(indAcierto))/len(yTrainS)

diferencia=yValE-yVal
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
PaVal=(100.0*len(indAcierto))/len(yVal)

diferencia=yTestE-yTest
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
indFallo = [i for (i, val) in enumerate(diferencia) if val != 0]
PaTest=(100.0*len(indAcierto))/len(yTest)

print("Probabilidad de acierto (Entrenamiento): %s"%(PaTrain))
print("Probabilidad de acierto (Validación): %s"%(PaVal))
print("Probabilidad de acierto (Test): %s"%(PaTest))

<font size=4, color=blue> Imágenes reconstruidas a partir de las representaciones obtenidas por los distintos autocodificadores

In [ ]:
# La reconstrucción del primer autocodificador se podría obtener directamente
#(xTrainDrec1,xTrainDE1)=mdl.mlp(xTrainS,[we1,wr1],tActAE1)
#xTrainDE1=xTrainDE1[0]
xTrainDE1=mdl.mlp(xTrainD,[we1,wr1],tActAE1)[1][0]
xTrainDE2=mdl.mlp(xTrainDE1,[we2,wr2],tActAE2)[1][0]
xTrainDE3=mdl.mlp(xTrainDE2,[we3,wr3],tActAE3)[1][0]

xTrainDrec1=mdl.mlp(xTrainDE1,[wr1],[tActAE1[1]])[0]
xTrainDrec2=mdl.mlp(xTrainDE2,[wr2,wr1],[tActAE2[1],tActAE1[1]])[0]
xTrainDrec3=mdl.mlp(xTrainDE3,[wr3,wr2,wr1],[tActAE3[1],tActAE2[1],tActAE1[1]])[0]

indices=[]

for k in range(10):
    indNum = [i for (i, val) in enumerate(yTrain[0:30]) if val == int(k)]
    indices.append(indNum[0])

figA, ejesA = plt.subplots(1, 10, figsize=(12, 3), subplot_kw={'xticks': [], 'yticks': []})
figA.subplots_adjust(hspace=0.3, wspace=0.05)
figB, ejesB = plt.subplots(1, 10, figsize=(12, 3), subplot_kw={'xticks': [], 'yticks': []})
figB.subplots_adjust(hspace=0.3, wspace=0.05)
figC, ejesC = plt.subplots(1, 10, figsize=(12, 3), subplot_kw={'xticks': [], 'yticks': []})
figC.subplots_adjust(hspace=0.3, wspace=0.05)
figD, ejesD = plt.subplots(1, 10, figsize=(12, 3), subplot_kw={'xticks': [], 'yticks': []})
figD.subplots_adjust(hspace=0.3, wspace=0.05)

for eje, k in zip(ejesA.flat, indices):
    imagen = xTrainD[:,k]
    imagen=imagen.reshape(len(pixeles),len(pixeles))
    etiqueta = yTrain[k]
    eje.imshow(imagen, cmap='gray')
    eje.set_title('a')
    eje.set_title('{label}'.format(label=etiqueta))
    
for eje, k in zip(ejesB.flat, indices):    
    imagen = xTrainDrec1[:,k]
    imagen=imagen.reshape(len(pixeles),len(pixeles))
    eje.imshow(imagen, cmap='gray') 
    
for eje, k in zip(ejesC.flat, indices):    
    imagen = xTrainDrec2[:,k]
    imagen=imagen.reshape(len(pixeles),len(pixeles))
    eje.imshow(imagen, cmap='gray')
    
for eje, k in zip(ejesD.flat, indices):    
    imagen = xTrainDrec3[:,k]
    imagen=imagen.reshape(len(pixeles),len(pixeles))
    eje.imshow(imagen, cmap='gray')


<br>
<font size=6, color=green>Red neuronal profunda con Drop-Out
<font size=4, color=blue>Se entrenará ahora una red neuronal de varias capas ocultas con inicialización aleatoria, Drop-Out, y funciones de activación ReLU (Rectified Linear Units)

In [ ]:
# Se comienza por definir los parámetros básicos del sistema
fCoste='entropia'        # Tipo de función de coste ('mmse', 'entropia' ,'wmmse')
opt='gradiente'          # Tipo de optimizador ('gradiente', 'momento')
tAct=[3,3,3,3,4]         # Tipos de activación de las neuronas de las capas oculta y de la capa de salida
paso=[1e-3]              # Parametro de paso de la actualización por gradiente
Nn1=125                  # Número de neuronas de la primera capa oculta
Nn2=125                  # Número de neuronas de la segunda capa oculta
Nn3=125                  # Número de neuronas de la tercera capa oculta 
Nn4=125                  # Número de neuronas de la tercera capa oculta 
Niter=500                # Número de iteraciones para la actualización por gradiente
Nbatch=250               # Número de muestras del mini-batch 
Nep=[Niter,Nbatch]
pDO=[0.1,0.3,0.3,0.3,0.3]  # Probabilidades de Drop-Out (de la entrada y de las capas ocultas)

# Se inicializan los pesos
Wini = mdl.entrena_mlp(xTrainS,eTrainS,[Nn1,Nn2,Nn3,Nn4,[]],[0,0])[0]
W=list(Wini)

In [ ]:
#Niter=2000
#Nep=[Niter,250]
(W, paso, coste) = mdl.entrena_mlp(xTrainS,eTrainS,W,Nep,fCoste=fCoste,
                                   optimizador=opt,tAct=tAct,paso=paso,pDO=pDO)[0:3]

font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
plt.plot(range(Niter+1),coste)
plt.title('Evolucion del coste a minimizar', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('Coste : %s' % (fCoste), fontdict=font)

<font size=4, color=blue> Se estiman las prestaciones obtenidas con esta red (con Drop-Out)

In [ ]:
eTrain=mdl.mlp(xTrainS,W,tAct)[0]
eVal=mdl.mlp(xValD,W,tAct)[0]
eTest=mdl.mlp(xTestD,W,tAct)[0]

yTrainE=eTrain.argmax(axis=0)
yValE=eVal.argmax(axis=0)
yTestE=eTest.argmax(axis=0)

diferencia=yTrainE-yTrainS
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
PaTrain=(100.0*len(indAcierto))/len(yTrainS)

diferencia=yValE-yVal
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
PaVal=(100.0*len(indAcierto))/len(yVal)

diferencia=yTestE-yTest
indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
indFallo = [i for (i, val) in enumerate(diferencia) if val != 0]
PaTest=(100.0*len(indAcierto))/len(yTest)

print("Probabilidad de acierto (Entrenamiento): %s"%(PaTrain))
print("Probabilidad de acierto (Validación): %s"%(PaVal))
print("Probabilidad de acierto (Test): %s"%(PaTest))

<font size=4, color=blue> Se representa la evolución de la tasa de acierto con las iteraciones

In [ ]:
W=list(Wini)
pEnt=0.1
pOcu=0.25
pDO=[pEnt,pOcu,pOcu,pOcu,pOcu]
NiterPaso=50
NrepBucle=100
evoPa=np.zeros((3,NrepBucle))
for kbucle in range(NrepBucle):
    #(W, paso, coste) = mdl.entrena_mlp(xTrainS,eTrainS,W,[NiterPaso,Nbatch],fCoste=fCoste,optimizador=opt,tAct=tAct)[0:3]
    (W, paso, coste) = mdl.entrena_mlp(xTrainS,eTrainS,W,[NiterPaso,Nbatch],fCoste=fCoste,
                                   optimizador=opt,tAct=tAct,paso=paso,pDO=pDO)[0:3]
    (eTrain,xOculta)=mdl.mlp(xTrainS,W,tAct)
    (eVal,xOculta)=mdl.mlp(xValD,W,tAct)
    (eTest,xOculta)=mdl.mlp(xTestD,W,tAct)

    yTrainE=eTrain.argmax(axis=0)
    yValE=eVal.argmax(axis=0)
    yTestE=eTest.argmax(axis=0)

    diferencia=yTrainE-yTrainS
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    PaTrain=(100.0*len(indAcierto))/len(yTrainS)

    diferencia=yValE-yVal
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    PaVal=(100.0*len(indAcierto))/len(yVal)

    diferencia=yTestE-yTest
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    indFallo = [i for (i, val) in enumerate(diferencia) if val != 0]
    PaTest=(100.0*len(indAcierto))/len(yTest)
    evoPa[:,kbucle]=[PaTrain,PaVal,PaTest]
    
font = {'family': 'serif','color':  'darkred','weight': 'normal','size': 16}
ejeIter=NiterPaso*np.linspace(1,NrepBucle,NrepBucle)
plt.plot(ejeIter,evoPa[0,:],ejeIter,evoPa[1,:],ejeIter,evoPa[2,:])
plt.title('Evolucion de la Prob. Acierto', fontdict=font)
plt.xlabel('Iteracion', fontdict=font)
plt.ylabel('Prob Acierto (%)', fontdict=font)

print("Entrenamiento (azul) - Validación (Naranja) - Test (verde)")
print(" ")
print("Max. Pa (Entrenamiento): %3.2f (Iter. %3.2f)"%(np.max(evoPa[0,:]),NiterPaso*np.argmax(evoPa[0,:])))
print("Max. Pa (Validación): %3.2f (Iter. %3.2f)"%(np.max(evoPa[1,:]),NiterPaso*np.argmax(evoPa[1,:])))
print("Max. Pa (Test) : %3.2f (Iter. %3.2f)"%(np.max(evoPa[2,:]),NiterPaso*np.argmax(evoPa[2,:])))
print("Resultado con validación (Test): %s"%(evoPa[2,np.argmax(evoPa[1,:])]))

<br>
<font size=6, color=green>Red neuronal convolucional (CNN)
<font size=4, color=blue>Se entrenará ahora una red neuronal convolucional

<font size=4, color=blue>Definición de la función para el entrenamiento de la red CNN usando TensorFlow

In [ ]:
# Se define la función básica para el entrenamiento de la red CNN usando TensorFlow
import tensorflow as tf

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def entrenaConv(xTrain,yTrain,W,Nepoch=[500,250],paso=1e-3,pDO=0.2,
                Npix=28,Npix2=7,Nconv1=5,Nconv2=5,Nfilt1=16,Nfilt2=32,
                Nneu=1024,padding=['SAME','SAME']):
    
    Niter = Nepoch[0]
    Nbatch = Nepoch[1]               
    (Ne,Np)=np.shape(xTrain)
    (Ns,Np)=np.shape(yTrain)
    ordenBatch = np.random.permutation(Np)
    Nini=0
    pDOkeep=1-pDO
    #--------------------------------------------------------------------------
    # Construcción de la red MLP Convolucional con Drop-out - Tensores              
    #--------------------------------------------------------------------------
    x = tf.placeholder(tf.float32, shape=[None, Ne])
    y = tf.placeholder(tf.float32, shape=[None, Ns])
    DO_keep_prob = tf.placeholder(tf.float32)
    pasoT = tf.placeholder(tf.float32)
    #--------------------------------------------------------------------------
    # Inicialización de pesos
    #--------------------------------------------------------------------------
    x_image = tf.reshape(x, [-1, Npix, Npix, 1])
    
    W_conv1 = weight_variable([Nconv1, Nconv1, 1, Nfilt1])
    b_conv1 = bias_variable([Nfilt1])
    
    W_conv2 = weight_variable([Nconv2, Nconv2, Nfilt1, Nfilt2])
    b_conv2 = bias_variable([Nfilt2])
    
    W_fc1 = weight_variable([Npix2 * Npix2 * Nfilt2, Nneu])
    b_fc1 = bias_variable([Nneu])    
           
    W_fc2 = weight_variable([Nneu, Ns])
    b_fc2 = bias_variable([Ns])    
    #--------------------------------------------------------------------------
    # Se propaga la red
    #--------------------------------------------------------------------------
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, 
                        strides=[1, 1, 1, 1], padding=padding[0])+b_conv1)
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding=padding[0])
    
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, 
                        strides=[1, 1, 1, 1], padding=padding[1])+b_conv2)
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding=padding[1])
        
    h_pool2_flat = tf.reshape(h_pool2, [-1, Npix2*Npix2*Nfilt2])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)        
    h_fc1_drop = tf.nn.dropout(h_fc1, DO_keep_prob)
    
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
                   
    coste_CONV = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_conv))
    #coste_CONV = tf.reduce_mean(tf.squared_difference(y,y_conv))
          
    entreno = tf.train.AdamOptimizer(learning_rate = pasoT).minimize(coste_CONV)     
    #----------------------------------------------------------------------------
    # Training - Se lanza la sesión
    #----------------------------------------------------------------------------
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    if len(W)>0:
        Wa = tf.assign(W_conv1,W[0])
        Wb = tf.assign(b_conv1,W[1])
        Wc = tf.assign(W_conv2,W[2])
        Wd = tf.assign(b_conv2,W[3])
        We = tf.assign(W_fc1,W[4])
        Wf = tf.assign(b_fc1,W[5])
        Wg = tf.assign(W_fc2,W[6])
        Wh = tf.assign(b_fc2,W[7])
        sess.run([Wa,Wb,Wc,Wd])
        sess.run([We,Wf,Wg,Wh])
            
    # Training loop
    for epoch in range(Niter+1):
        if (Nini+Nbatch) > Np:
            ordenBatch = np.random.permutation(Np)        
            indBatch=ordenBatch[np.arange(Nbatch)]
            Nini=Nbatch
        else:        
            indBatch=ordenBatch[Nini+np.arange(Nbatch)]
            Nini=Nini+Nbatch
            
        xTrainBatch=xTrain[:,indBatch]
        yTrainBatch=yTrain[:,indBatch]

        # Entreno con los datos del BATCH
        sess.run(entreno, feed_dict={x: xTrainBatch.T, y: yTrainBatch.T, pasoT: paso, DO_keep_prob: pDOkeep})        

    
    (a,b,c,d)=sess.run([W_conv1,b_conv1,W_conv2,b_conv2])
    (e,f,g,h)=sess.run([W_fc1,b_fc1,W_fc2,b_fc2])
    Wtodo=[a,b,c,d,e,f,g,h]

    sess.close()
    pRed=[Npix,Npix2,Nconv1,Nconv2,Nfilt1,Nfilt2,Nneu,Ns,padding]
    return (Wtodo,pRed)

<font size=4, color=blue>Definición de la función evaluar la salida de la red CNN

In [ ]:
def evaluaConv(xTrain,Wtodo,pRed=[28,7,5,5,32,64,1024,10,['SAME','SAME']]):
    
    (Ne,Np)=np.shape(xTrain)
    #--------------------------------------------------------------------------
    # Tensores y definición de parámetros básicos
    #--------------------------------------------------------------------------
    x = tf.placeholder(tf.float32, shape=[None, Ne])
    DO_keep_prob = tf.placeholder(tf.float32)
    
    Npix=pRed[0]
    Npix2=pRed[1]
    Nconv1=pRed[2]
    Nconv2=pRed[3]
    Nfilt1=pRed[4]
    Nfilt2=pRed[5]   
    Nneu=pRed[6]
    Ns=pRed[7]
    padding=pRed[8]
    
    #--------------------------------------------------------------------------
    # Inicialización de pesos
    #--------------------------------------------------------------------------
    x_image = tf.reshape(x, [-1, Npix, Npix, 1])
    
    W_conv1 = weight_variable([Nconv1, Nconv1, 1, Nfilt1])
    b_conv1 = bias_variable([Nfilt1])
    
    W_conv2 = weight_variable([Nconv2, Nconv2, Nfilt1, Nfilt2])
    b_conv2 = bias_variable([Nfilt2])
    
    W_fc1 = weight_variable([Npix2 * Npix2 * Nfilt2, Nneu])
    b_fc1 = bias_variable([Nneu])    
           
    W_fc2 = weight_variable([Nneu, Ns])
    b_fc2 = bias_variable([Ns])    
    #--------------------------------------------------------------------------
    # Se propaga la red
    #--------------------------------------------------------------------------
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, 
                        strides=[1, 1, 1, 1], padding=padding[0])+b_conv1)
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding=padding[0])
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, 
                        strides=[1, 1, 1, 1], padding=padding[1])+b_conv2)
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding=padding[1])
        
    h_pool2_flat = tf.reshape(h_pool2, [-1, Npix2*Npix2*Nfilt2])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)        
    h_fc1_drop = tf.nn.dropout(h_fc1, DO_keep_prob)
    
    
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2                   
    #------------------------------------------------------------------------------
    # Evaluación de la salida de la red - Se lanza la sesión
    #------------------------------------------------------------------------------
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    Wa = tf.assign(W_conv1,Wtodo[0])
    Wb = tf.assign(b_conv1,Wtodo[1])
    Wc = tf.assign(W_conv2,Wtodo[2])
    Wd = tf.assign(b_conv2,Wtodo[3])
    We = tf.assign(W_fc1,Wtodo[4])
    Wf = tf.assign(b_fc1,Wtodo[5])
    Wg = tf.assign(W_fc2,Wtodo[6])
    Wh = tf.assign(b_fc2,Wtodo[7])
    sess.run([Wa,Wb,Wc,Wd])
    sess.run([We,Wf,Wg,Wh])    
        
    eTrain = sess.run(y_conv, feed_dict={x: xTrain.T, DO_keep_prob: 1.0})

    sess.close()
    return (eTrain.T)

<font size=4, color=blue>Definición de los parámetros básicos de la red CNN

In [ ]:
Wcnn=[]        # Inicialización de pesos de la CNN 
Niter=25       # Número de iteraciones de cada paso 
Nbatch=250     # Número de muestras de cada mini-batch
Nreps=50       # Número de pasos de entrenamiento
pDO=0.2
evoRes=np.zeros((3,Nreps))
if xTrainS.shape[0]==100: # 10x10 píxeles
    Npix=10    # Número de píxeles por dirección        
    Nconv1=3   # Tamaño del filtro convolucional 1 (píxeles)
    Nconv2=3   # Tamaño del filtro convolucional 1 (píxeles) 
    padding=['VALID','SAME']  # Tipo de padding para las dos etapas
    Npix2=2    # Número de píxeles tras las dos etapas de Pooling
    Nf1=16     # Número de filtros de la primera etapa
    Nf2=32     # Número de filtros de la segunda etapa
    Nn=256     # Número de neuronas del MLP final
elif xTrainS.shape[0]==196: # 14x14 píxeles
    Npix=14    # Número de píxeles por dirección        
    Nconv1=3   # Tamaño del filtro convolucional 1 (píxeles)
    Nconv2=3   # Tamaño del filtro convolucional 1 (píxeles) 
    padding=['VALID','SAME']  # Tipo de padding para las dos etapas
    Npix2=3    # Número de píxeles tras las dos etapas de Pooling
    Nf1=32     # Número de filtros de la primera etapa
    Nf2=64     # Número de filtros de la segunda etapa
    Nn=1024    # Número de neuronas del MLP final
elif xTrainS.shape[0]==784: # 18x28 píxeles 
    Npix=28    # Número de píxeles por dirección        
    Nconv1=5   # Tamaño del filtro convolucional 1 (píxeles)
    Nconv2=5   # Tamaño del filtro convolucional 1 (píxeles) 
    padding=['SAME','SAME']  # Tipo de padding para las dos etapas
    Npix2=7    # Número de píxeles tras las dos etapas de Pooling
    Nf1=32     # Número de filtros de la primera etapa
    Nf2=64     # Número de filtros de la segunda etapa
    Nn=1024    # Número de neuronas del MLP final
    

<font size=4, color=blue>Entrenamiento iterativo de la red CNN

In [ ]:
for kRep in range(Nreps):
    (Wcnn,pRed)=entrenaConv(xTrainS,eTrainS,Wcnn,Nepoch=[Niter,Nbatch],paso=1e-3,
                 pDO=pDO,padding=padding,Npix=Npix,Npix2=Npix2,Nconv1=Nconv1,Nconv2=Nconv2,
                 Nfilt1=Nf1,Nfilt2=Nf2,Nneu=Nn)
        
    eTrain=evaluaConv(xTrainS,Wcnn,pRed)
    eVal=evaluaConv(xValD,Wcnn,pRed)
    eTest=evaluaConv(xTestD,Wcnn,pRed)
        
    yTrainE=eTrain.argmax(axis=0)
    yValE=eVal.argmax(axis=0)
    yTestE=eTest.argmax(axis=0)
        
    diferencia=yTrainE-yTrainS
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    PaTrain=(100.0*len(indAcierto))/len(yTrainS)
        
    diferencia=yValE-yVal
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    PaVal=(100.0*len(indAcierto))/len(yVal)
        
    diferencia=yTestE-yTest
    indAcierto = [i for (i, val) in enumerate(diferencia) if val == 0]
    indFallo = [i for (i, val) in enumerate(diferencia) if val != 0]
    PaTest=(100.0*len(indAcierto))/len(yTest)
        
    evoRes[:,kRep]=[PaTrain,PaVal,PaTest]
        
    print("Repetición %d de %d"%(kRep+1,Nreps))
    print("Probabilidad de acierto (Entrenamiento): %s"%(PaTrain))
    print("Probabilidad de acierto (Validación): %s"%(PaVal))
    print("Probabilidad de acierto (Test): %s"%(PaTest))
    print("--------------------------------------")

In [ ]:
mdl=reload(mdl)